### Install dependencies


In [ ]:
!pip install onnx
!pip install onnxruntime

import torch.onnx
import onnx
import onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import copy
import time


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.utils.prune as prune
from torchsummary import summary


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
def load_model(path):
    model = models.resnet18()
    model.fc = nn.Linear(512, 100)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    return model


model = load_model(path = '/content/drive/MyDrive/UCI/ResNet18_CIFAR100.pth')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Export onnx file

In [ ]:
model.eval()
model.to(device)

input_data = torch.randn(1, 3, 32, 32).to(device)  # model input_size

# export model to '.onnx' format
torch.onnx.export(model, input_data, "output2.onnx", operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

print("Successful")

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Successful


### Load 'onnx' file

In [ ]:
# load the onnx format file
onnx_model = onnx.load('/content/drive/MyDrive/UCI/output2.onnx')  # from model_compression.ipynb

In [ ]:
# test for 'onnx' format file

onnx_model.graph.node[0].input
onnx_model.graph.node[0].output

['/conv1/Conv_output_0']

### Building Graph

In [ ]:
# Node class definition
class Node:
    def __init__(self, name, node_type, inputs, outputs):
        self.name = name  # str
        self.node_type = node_type  # str
        self.inputs = inputs  # List[str]
        self.outputs = outputs  # List[str]
        self.parent = []  # parent node names
        self.child_nodes = []  # child node names

    def __repr__(self):
        return f"node_type={self.node_type}, parent={', '.join(self.parent)}, child_nodes={', '.join(self.child_nodes)}"


In [ ]:
node_dict = {}

# Create all the nodes from onnx file
for node in onnx_model.graph.node:
    node_name = node.name
    node_type = node.op_type
    node_inputs = list(node.input)
    node_outputs = list(node.output)

    node_dict[node_name] = Node(node_name, node_type, node_inputs, node_outputs)  # create Node instance

# Find dependencies each other
for node in onnx_model.graph.node:
    current_node = node_dict[node.name]
    # Check one's output with other's input
    for output_name in node.output:
        for another_node in onnx_model.graph.node:
            if output_name in another_node.input:
                # If matched
                if another_node.name in node_dict:
                    # append to the list
                    current_node.child_nodes.append(another_node.name)  # for child nodes
                    node_dict[another_node.name].parent.append(current_node.name)  # for parent nodes

node_dict  # key : node_name / value : Node instance

{'/conv1/Conv': node_type=Conv, parent=, child_nodes=/relu/Relu,
 '/relu/Relu': node_type=Relu, parent=/conv1/Conv, child_nodes=/maxpool/MaxPool,
 '/maxpool/MaxPool': node_type=MaxPool, parent=/relu/Relu, child_nodes=/layer1/layer1.0/conv1/Conv, /layer1/layer1.0/Add,
 '/layer1/layer1.0/conv1/Conv': node_type=Conv, parent=/maxpool/MaxPool, child_nodes=/layer1/layer1.0/relu/Relu,
 '/layer1/layer1.0/relu/Relu': node_type=Relu, parent=/layer1/layer1.0/conv1/Conv, child_nodes=/layer1/layer1.0/conv2/Conv,
 '/layer1/layer1.0/conv2/Conv': node_type=Conv, parent=/layer1/layer1.0/relu/Relu, child_nodes=/layer1/layer1.0/Add,
 '/layer1/layer1.0/Add': node_type=Add, parent=/maxpool/MaxPool, /layer1/layer1.0/conv2/Conv, child_nodes=/layer1/layer1.0/relu_1/Relu,
 '/layer1/layer1.0/relu_1/Relu': node_type=Relu, parent=/layer1/layer1.0/Add, child_nodes=/layer1/layer1.1/conv1/Conv, /layer1/layer1.1/Add,
 '/layer1/layer1.1/conv1/Conv': node_type=Conv, parent=/layer1/layer1.0/relu_1/Relu, child_nodes=/lay

Just test

In [ ]:
att = ['name', 'node_type', 'inputs', 'outputs', 'parent', 'child_nodes']  # define in the Node class

# test for Conv0
for node in att:
    print(f"{node}: {getattr(node_dict[onnx_model.graph.node[0].name], node)}")

name: /conv1/Conv
node_type: Conv
inputs: ['input.1', 'onnx::Conv_193', 'onnx::Conv_194']
outputs: ['/conv1/Conv_output_0']
parent: []
child_nodes: ['/relu/Relu']


In [ ]:
# Node class definition
# class Node:
#     def __init__(self, name, node_type):
#         self.name = name  # str
#         self.node_type = node_type  # str
#         self.parent = [] # parent node, list of nodes if there are multiple inputs
#         self.child_nodes = []

#     def __repr__(self):
#         return f"node_type={self.node_type}, parent={', '.join([node.name for node in self.parent])}, child_nodes={', '.join([node.name for node in self.child_nodes])}"

# node_dict = {}  # key : node_name

# # First, create all the nodes
# for node in onnx_model.graph.node:
#     node_name = node.output[0]  # use the first output name as the node name
#     node_type = node.op_type  # str
#     node_dict[node_name] = Node(node_name, node_type)

# # Next, link the nodes together
# for node in onnx_model.graph.node:
#     current_node = node_dict[node.output[0]]  # use the first output name as the node name
#     for input_name in node.input:
#         if input_name in node_dict:
#             parent_node = node_dict[input_name]
#             parent_node.child_nodes.append(current_node)
#             current_node.parent.append(parent_node)

# # print node_dict
# node_dict
